# Installation and Mounting

In [1]:
# !pip install transformers==4.22.2
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.8 MB/s 


In [2]:
from PIL import Image
import requests
import torch
# from transformers import CLIPProcessor, CLIPModel
import pandas as pd
import numpy as np

In [4]:
git_token = "Your Persoanl Access Token in Georgia Tech Github"

In [5]:
!git clone https://{git_token}@github.gatech.edu/tpang34/XLM_CLIP.git

Cloning into 'XLM_CLIP'...
remote: Enumerating objects: 3117, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 3117 (delta 20), reused 31 (delta 13), pack-reused 3072
Receiving objects: 100% (3117/3117), 11.26 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (824/824), done.


In [6]:
!pip install /content/XLM_CLIP/transformers_mod/.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./XLM_CLIP/transformers_mod
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 7.0 MB/s 
     |████████████████████████████████| 7.6 MB 71.2 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5711750 sha256=379c5d66bfed4e8cbde6845f5feb3bb996c23a9789cc1dacc17f6f88e5100c06
  Stored in directory: /tmp/pip-ephem-wheel-cache-_banu

In [ ]:
from transformers import CLIPModel

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

In [ ]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
device = 'cuda'
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")# .to(device)


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CLIPModel were not initialized from the model checkpoint at openai/clip-vit-base-patch32 and are newly initialized: ['xlm_encoder.encoder.layer.22.attention.self.query.weight', 'xlm_encoder.encoder.layer.20.output.dense.bias', 'xlm_encoder.encoder.layer.23.attention.

In [ ]:

url1 = "http://images.cocodataset.org/val2017/000000039769.jpg"
image1 = Image.open(requests.get(url1, stream=True).raw)

url2 = "http://images.cocodataset.org/test-stuff2017/000000002383.jpg"
image2 = Image.open(requests.get(url2, stream=True).raw)

text1 =  "a photo of a cat" # "una foto de un gato" 
text2 = "a photo of two zebra"

xlm_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")# .to(device)
xlm_inputs = xlm_tokenizer([text1], return_tensors="pt").to(device)

clip_inputs = clip_processor(
    text=[text1], images=[image1, image2, image1], return_tensors="pt", padding=True
).to(device)



In [ ]:
outputs = clip_model(xlm_inputs=xlm_inputs, **clip_inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=0)  # we can take the softmax to get the label probabilities


In [ ]:
probs

tensor([[0.4990],
        [0.0019],
        [0.4990]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# Download Data

In [ ]:
# Download MS-COCO-Spanish
!git clone https://github.com/carlosgarciahe/ms-coco-es

/content
fatal: destination path 'ms-coco-es' already exists and is not an empty directory.


# Training

In [ ]:
# freezing all other layers
for param in clip_model.parameters():
    param.requires_grad = False
clip_model.agg_text_projection.requires_grad_(True)

Linear(in_features=1536, out_features=512, bias=False)

In [ ]:
optimizer_config = {
  "optimizer_type": "AdamW",
  "lr": 1e-2,
  "weight_decay": 0,
  "momentum": None}

In [ ]:
optimizer = get_optimizer(clip_model, optimizer_config)

In [ ]:
data_dir = "/content/ms-coco-es"
model_base_path = 'clip/'

In [ ]:
trainer = Trainer(data_dir= data_dir, 
                  model = clip_model,
                  optimizer = optimizer,
                  model_dir = model_base_path,
                  load_from_disk = False,
                  cuda = True
                 )

In [ ]:
trainer.train(num_epochs=3)

Epoch:0, Training Loss:2.4402, Validation Loss:2.3874


RuntimeError: ignored

In [ ]:
trainer.train_loss_list

In [ ]:
trainer.plot_loss()
trainer.plot_accuracy()
trainer.plot_recall5()
trainer.plot_recall10()